In [1]:
%cd ../data

/home/philippe/MolGenDocking/data


/home/philippe/miniconda3/envs/MolGen/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from huggingface_hub import HfApi

In [5]:
DATASET_NAMES = ["molgendata", "polaris", "synthesis", "analog_gen"]

In [6]:
import os
import time

api = HfApi()
for name in DATASET_NAMES[:]:
    os.system("rm {}.tar.gz".format(name))
    print("Deleted '{}', compressing again".format(name))
    t0 = time.time()
    os.system("tar -czf {}.tar.gz {}".format(name, name))
    t1 = time.time()
    print("Compressed '{}' in {:.2f}s".format(name, t1-t0))
    api.upload_file(
        path_or_fileobj="{}.tar.gz".format(name),
        path_in_repo="{}.tar.gz".format(name),
        repo_id="Franso/MolGenData",
        repo_type="dataset",
    )
    t2 = time.time()
    print("Uploaded '{}' in {:.2f}s".format(name, t2-t1))



Deleted 'polaris', compressing again
Compressed 'polaris' in 0.99s


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Uploaded 'polaris' in 1.47s
